# Lab 2: Azure AI Search RAG 지식 베이스 구성하기

## 개요 (Overview)

이 노트북에서는 Azure AI Search를 사용하여 RAG(Retrieval-Augmented Generation) 지식 베이스를 구성합니다.

### 아키텍처 (Architecture)

```
┌─────────────────────────────────────────────────────────┐
│                  RAG Pipeline                            │
│                                                          │
│  Documents (JSON) → Embeddings (OpenAI)                 │
│                         ↓                                │
│              Azure AI Search Index                       │
│           (Vector Store + Keyword Search)                │
│                         ↓                                │
│            Hybrid Search (Vector + BM25)                 │
│                         ↓                                │
│              Research Agent Query                        │
└─────────────────────────────────────────────────────────┘
```

### 학습 목표 (Learning Objectives)

이 실습을 완료하면 다음을 할 수 있게 됩니다:

1. ✅ Azure AI Search 인덱스 스키마 설계 및 생성
2. ✅ Azure OpenAI로 텍스트 임베딩 생성
3. ✅ 벡터 및 키워드 검색을 위한 문서 업로드
4. ✅ 하이브리드 검색 (벡터 + BM25) 실행 및 테스트
5. ✅ RAG 파이프라인 성능 평가 및 최적화

### 사용할 데이터 (Data)

- **data/knowledge-base.json**: 54개의 AI 에이전트 관련 문서 (섹션별 청킹)
- **카테고리**: Agent Development, RAG Patterns, MCP, Deployment, Architecture
- **임베딩 모델**: text-embedding-3-large (3072차원 기본값 사용)

## 1. 사전 요구 사항 확인 (Prerequisites Check)

다음 도구들이 설치되어 있는지 확인합니다:

- Python 3.9 이상
- Azure CLI
- Azure Developer CLI (azd)
- Docker (Container Apps 배포 시 필요)

In [1]:
import sys, subprocess, os
import platform

# 운영체제에 따라 PATH 설정 (macOS, Linux, Codespaces 모두 지원)
system = platform.system()
if system == 'Darwin':  # macOS
    # Homebrew 경로 추가 (Intel & Apple Silicon)
    extra_paths = '/opt/homebrew/bin:/usr/local/bin'
elif system == 'Linux':  # Linux / Codespaces
    # 일반적인 Linux 바이너리 경로
    extra_paths = '/usr/local/bin:/usr/bin:/home/codespace/.local/bin'
else:  # Windows
    extra_paths = ''

if extra_paths:
    os.environ['PATH'] = extra_paths + ':' + os.environ.get('PATH', '')

def check(cmd, name):
    try:
        result = subprocess.run(cmd, shell=True, capture_output=True, timeout=3, env=os.environ)
        print(f"{'✓' if result.returncode == 0 else '✗'} {name}")
    except Exception as e:
        print(f"✗ {name}")

print("=== Prerequisites Check ===")
print(f"✓ Python {sys.version.split()[0]} ({system})")
check("az --version", "Azure CLI")
check("azd version", "Azure Developer CLI")
check("docker --version", "Docker")
print("="*50)

=== Prerequisites Check ===
✓ Python 3.13.7 (Darwin)
✓ Azure CLI
✓ Azure Developer CLI
✓ Docker


## 2. 패키지 설치 및 설정 로드 (Install Packages & Load Configuration)

In [2]:
# 필요한 패키지 설치
import sys
import subprocess

packages = [
    "azure-search-documents>=11.4.0",
    "azure-identity>=1.15.0",
    "openai>=1.12.0",
    "python-dotenv>=1.0.0"
]

print("📦 패키지 설치 중...")
for package in packages:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", package])
print("✅ 패키지 설치 완료")

📦 패키지 설치 중...
✅ 패키지 설치 완료


In [3]:
# Notebook 1에서 저장한 설정 파일 로드
import json
import os

config_path = "./config.json"

if not os.path.exists(config_path):
    raise FileNotFoundError(
        f"❌ 설정 파일을 찾을 수 없습니다: {config_path}\n"
        "먼저 Notebook 1 (01_deploy_azure_resources.ipynb)을 실행하세요."
    )

with open(config_path, 'r') as f:
    config = json.load(f)

# 필수 설정 확인 (Managed Identity 사용으로 키 불필요)
required_keys = ["search_endpoint", "project_connection_string"]
missing_keys = [key for key in required_keys if not config.get(key)]

if missing_keys:
    raise ValueError(f"❌ 필수 설정이 누락되었습니다: {', '.join(missing_keys)}")

print("✅ 설정 파일 로드 완료")
print(f"📍 Search Endpoint: {config['search_endpoint']}")
print(f"📍 AI Project Connection: {'✓ Set' if config['project_connection_string'] else '✗ Missing'}")
print(f"🔐 인증 방식: Managed Identity (DefaultAzureCredential)")

✅ 설정 파일 로드 완료
📍 Search Endpoint: https://srch-flyoy4n4dll42.search.windows.net/
📍 AI Project Connection: ✓ Set
🔐 인증 방식: Managed Identity (DefaultAzureCredential)


## 3. Azure 인증 (Azure Authentication)

Lab 1에서 이미 Azure에 로그인했지만, 세션이 만료되었을 수 있으므로 인증 상태를 확인하고 필요시 재인증합니다.

In [ ]:
import subprocess, json

print("=== Azure Authentication ===")
print("ℹ️  인증 상태를 확인하고 필요시 로그인합니다.\n")

# Azure CLI 인증 상태 확인
az_account = subprocess.run("az account show", shell=True, capture_output=True, text=True)

if az_account.returncode == 0:
    account_info = json.loads(az_account.stdout)
    print(f"✅ Azure CLI 인증 완료 (기존 세션 사용)")
    print(f"   구독: {account_info.get('name', 'N/A')}")
    print(f"   테넌트: {account_info.get('tenantId', 'N/A')}")
else:
    print("⚠️  Azure CLI 인증이 필요합니다. 브라우저가 열립니다...")
    az_login = subprocess.run("az login --tenant 16b3c013-d300-468d-ac64-7eda0820b6d3", shell=True)
    if az_login.returncode == 0:
        print("✅ Azure CLI 로그인 완료")
    else:
        raise Exception("❌ Azure CLI 로그인 실패")

print("="*50)

=== Azure Authentication ===
🔐 Browser will open for authentication...

✅ Azure CLI
✅ Azure Developer CLI


## 4. 지식 베이스 데이터 로드 (Load Knowledge Base Data)

In [5]:
# 지식 베이스 JSON 파일 로드
knowledge_base_path = "./data/knowledge-base.json"

if not os.path.exists(knowledge_base_path):
    raise FileNotFoundError(f"❌ 지식 베이스 파일을 찾을 수 없습니다: {knowledge_base_path}")

with open(knowledge_base_path, 'r', encoding='utf-8') as f:
    knowledge_base = json.load(f)

# JSON이 직접 배열이거나 documents 래퍼가 있을 수 있음
if isinstance(knowledge_base, list):
    documents = knowledge_base
else:
    documents = knowledge_base.get("documents", [])

print(f"✅ 지식 베이스 로드 완료")
print(f"📚 총 문서 개수: {len(documents)}")
print(f"\n📂 카테고리별 문서 수:")

# 카테고리별 분류
from collections import Counter
categories = Counter(doc["category"] for doc in documents)
for category, count in categories.items():
    print(f"  • {category}: {count}개")

# 첫 번째 문서 샘플 출력
if documents:
    print(f"\n📄 샘플 문서:")
    sample = documents[0]
    print(f"  ID: {sample['id']}")
    print(f"  제목: {sample['title']}")
    print(f"  카테고리: {sample['category']}")
    print(f"  섹션: {sample.get('section', 'N/A')}")
    print(f"  내용 길이: {len(sample['content'])} 자")


✅ 지식 베이스 로드 완료
📚 총 문서 개수: 54

📂 카테고리별 문서 수:
  • 에이전트 개발 기초: 7개
  • 멀티 에이전트 아키텍처: 9개
  • RAG와 Azure AI Search: 6개
  • 사용자 쿼리: 1개
  • 지식 베이스 검색: 1개
  • 컨텍스트 구성: 1개
  • 에이전트에 컨텍스트 제공: 1개
  • 에이전트 실행: 4개
  • Model Context Protocol (MCP): 11개
  • Container Apps 배포: 13개

📄 샘플 문서:
  ID: doc-01-00
  제목: 에이전트 개발 기초 - 개요
  카테고리: 에이전트 개발 기초
  섹션: 1
  내용 길이: 176 자


## 5. Azure AI Search 인덱스 생성 (Create Search Index)

### 📝 인덱서(Indexer) vs. 직접 업로드 방식

이 Lab에서는 **인덱서(Indexer)를 사용하지 않고** 직접 문서를 업로드하는 방식을 사용합니다.

**두 방식의 차이:**

| 구분 | 인덱서 방식 | 직접 업로드 방식 (이 Lab) |
|------|------------|------------------------|
| **데이터 소스** | Blob Storage, Cosmos DB 등 | JSON 파일, 메모리 데이터 |
| **자동화** | 자동 크롤링 및 업데이트 | 수동 업로드 (코드 실행 시) |
| **임베딩** | Azure OpenAI Skills 사용 | Python 코드로 직접 생성 |
| **복잡도** | 높음 (여러 리소스 필요) | 낮음 (코드만 있으면 됨) |
| **유연성** | 제한적 | 높음 (완전한 제어) |
| **적합한 경우** | 대량 데이터, 정기 업데이트 | 소량 데이터, 일회성 작업 |

**이 Lab에서 직접 업로드 방식을 선택한 이유:**
- ✅ 학습 목적: RAG 파이프라인의 각 단계를 명확히 이해
- ✅ 간단한 구성: 추가 Azure 리소스 불필요 (Blob Storage 등)
- ✅ 완전한 제어: 임베딩 생성 과정을 직접 확인 가능
- ✅ 빠른 테스트: 코드 실행만으로 즉시 업데이트 가능

---

### 🏗️ 인덱스 스키마 설계

인덱스 스키마는 검색 성능과 품질에 직접적인 영향을 미치는 중요한 설계 요소입니다.

#### 📋 필드 구성 및 역할

**1. `id` (Primary Key)**
```python
SimpleField(name="id", type=SearchFieldDataType.String, key=True)
```
- **역할**: 문서의 고유 식별자
- **타입**: String (예: "doc_001", "doc_002")
- **key=True**: 인덱스의 기본 키로 지정 (필수, 중복 불가)
- **사용 예시**: 문서 업데이트, 삭제 시 식별자로 사용

**2. `title` (제목)**
```python
SearchableField(name="title", type=SearchFieldDataType.String, 
               filterable=True, sortable=True)
```
- **역할**: 문서 제목 (검색 대상)
- **SearchableField**: 전문 검색(Full-text search) 가능
- **filterable=True**: 필터링 가능 (예: `title eq '특정 제목'`)
- **sortable=True**: 정렬 가능 (예: 제목 가나다순)
- **사용 예시**: 제목으로 검색, 정렬, 필터링

**3. `content` (본문) - 가장 중요한 필드**
```python
SearchableField(name="content", type=SearchFieldDataType.String, 
               analyzer_name="ko.microsoft")
```
- **역할**: 문서의 실제 내용 (주요 검색 대상)
- **analyzer_name="ko.microsoft"**: 한국어 분석기 사용 ⭐
- **왜 한국어 분석기를 사용하는가?**
  - 형태소 분석: "에이전트를" → "에이전트" (조사 제거)
  - 어간 추출: "개발하다", "개발하기" → "개발" (동일 개념)
  - 불용어 제거: "그리고", "그러나" 등 의미 없는 단어 제거
  - 검색 품질 향상: "RAG를" 검색 시 "RAG가", "RAG는" 모두 매칭
- **대안**: `ko.lucene` (Lucene 한국어 분석기, 더 가벼움)
- **영어 문서**: `en.microsoft` 또는 `en.lucene` 사용

**4. `category` (카테고리)**
```python
SimpleField(name="category", type=SearchFieldDataType.String, 
           filterable=True, sortable=True, facetable=True)
```
- **역할**: 문서 분류 (예: "Agent", "RAG", "MCP")
- **SimpleField**: 검색 대상 아님 (필터/정렬만)
- **filterable=True**: 카테고리별 필터링 가능
- **facetable=True**: 패싯 검색 가능 (카테고리별 문서 수 집계)
- **사용 예시**: 
  ```python
  # "Agent" 카테고리만 검색
  results = search_client.search("보안", filter="category eq 'Agent'")
  ```

**5. `section` (섹션)**
```python
SimpleField(name="section", type=SearchFieldDataType.String, 
           filterable=True, sortable=False)
```
- **역할**: 문서 내 섹션 구분 (예: "개발", "배포", "최적화")
- **filterable=True**: 섹션별 필터링 가능
- **sortable=False**: 정렬 불필요 (섹션 이름에 순서 의미 없음)

**6. `contentVector` (벡터 임베딩) - RAG의 핵심**
```python
SearchField(
    name="contentVector",
    type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
    searchable=True,
    vector_search_dimensions=3072,
    vector_search_profile_name="vector-profile"
)
```
- **역할**: 문서 내용의 벡터 표현 (의미 검색용)
- **Collection(Single)**: Float 배열 (3072개의 부동소수점 값)
- **vector_search_dimensions=3072**: 벡터 차원 수 ⭐
  - **왜 3072 차원인가?**
    - `text-embedding-3-large` 모델의 기본 출력 차원
    - 더 높은 차원 = 더 정확한 의미 표현
    - `text-embedding-3-small`은 1536 차원 (더 빠르지만 덜 정확)
    - `text-embedding-ada-002`는 1536 차원 (구 모델)
  - **차원 수와 성능**:
    - 높은 차원: 의미 구별 능력 향상, 검색 정확도 ↑, 저장 공간 ↑
    - 낮은 차원: 검색 속도 ↑, 저장 공간 ↓, 정확도 ↓
- **vector_search_profile_name**: HNSW 알고리즘 프로필 사용

---

#### 🔍 벡터 검색 알고리즘: HNSW

**HNSW (Hierarchical Navigable Small World)**
- Azure AI Search의 기본 벡터 검색 알고리즘
- 대규모 벡터 데이터에서 빠른 근사 최근접 이웃 검색
- 정확도 vs 속도 트레이드오프 조정 가능

**설정:**
```python
vector_search = VectorSearch(
    algorithms=[
        HnswAlgorithmConfiguration(name="hnsw-algorithm")
    ],
    profiles=[
        VectorSearchProfile(
            name="vector-profile",
            algorithm_configuration_name="hnsw-algorithm"
        )
    ]
)
```

**HNSW vs 다른 알고리즘:**
| 알고리즘 | 속도 | 정확도 | 메모리 | 적합한 경우 |
|---------|------|-------|--------|------------|
| **HNSW** | ⭐⭐⭐⭐ | ⭐⭐⭐⭐⭐ | 높음 | 일반적인 RAG (권장) |
| Exhaustive KNN | ⭐ | ⭐⭐⭐⭐⭐ | 낮음 | 소량 데이터, 최고 정확도 필요 |
| IVF | ⭐⭐⭐⭐⭐ | ⭐⭐⭐ | 낮음 | 대량 데이터, 속도 중시 |

---

#### 📊 필드 구성 요약

| 필드 | 타입 | 검색 | 필터 | 정렬 | 패싯 | 용도 |
|------|------|------|------|------|------|------|
| **id** | String | ❌ | ❌ | ❌ | ❌ | 문서 식별자 (Primary Key) |
| **title** | String | ✅ | ✅ | ✅ | ❌ | 제목 검색, 정렬 |
| **content** | String | ✅ | ❌ | ❌ | ❌ | 본문 키워드 검색 (BM25) |
| **category** | String | ❌ | ✅ | ✅ | ✅ | 카테고리 필터링, 집계 |
| **section** | String | ❌ | ✅ | ❌ | ❌ | 섹션 필터링 |
| **contentVector** | Float[] | ✅ | ❌ | ❌ | ❌ | 벡터 검색 (의미 기반) |

---

#### 💡 스키마 설계 모범 사례

**1. 최소한의 필드만 사용**
- 불필요한 필드는 검색 성능 저하
- 이 Lab: 6개 필드로 모든 기능 구현

**2. 적절한 필드 타입 선택**
- 검색 대상: `SearchableField` (title, content)
- 필터/정렬만: `SimpleField` (category, section)
- 벡터 검색: `SearchField` with vector config

**3. 분석기 선택**
- 한국어 문서: `ko.microsoft` (권장)
- 영어 문서: `en.microsoft`
- 다국어: 언어별 필드 분리 또는 `standard.lucene` (범용)

**4. 벡터 차원 선택**
- 정확도 우선: 3072 차원 (text-embedding-3-large)
- 속도 우선: 1536 차원 (text-embedding-3-small)
- 비용 고려: 차원 수 ↑ = 저장 비용 ↑

**5. 인덱스 재생성 주의**
- 스키마 변경 시 기존 인덱스 삭제 후 재생성 필요
- 프로덕션에서는 무중단 마이그레이션 전략 필요

In [8]:
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import (
    SearchIndex,
    SearchField,
    SearchFieldDataType,
    VectorSearch,
    VectorSearchProfile,
    HnswAlgorithmConfiguration,
    SimpleField,
    SearchableField
)
from azure.core.credentials import AzureKeyCredential
import subprocess

# Azure AI Search Admin Key 가져오기
print("🔑 AI Search Admin Key 가져오는 중...")
search_service_name = config.get("search_service_name", "")
resource_group = config.get("resource_group", "")

key_result = subprocess.run(
    f"az search admin-key show --resource-group {resource_group} --service-name {search_service_name} --query primaryKey -o tsv",
    shell=True,
    capture_output=True,
    text=True
)

if key_result.returncode != 0:
    raise Exception(f"❌ Admin Key를 가져올 수 없습니다: {key_result.stderr}")

search_admin_key = key_result.stdout.strip()
print("✅ Admin Key 획득 완료")

# Search Index Client 생성 (Admin Key 인증)
index_client = SearchIndexClient(
    endpoint=config["search_endpoint"],
    credential=AzureKeyCredential(search_admin_key)
)
print("✅ Search Index Client 생성 완료 (Admin Key 인증)")

# 인덱스 이름 설정
index_name = "ai-agent-knowledge-base"

# 인덱스 스키마 정의
fields = [
    SimpleField(name="id", type=SearchFieldDataType.String, key=True),
    SearchableField(name="title", type=SearchFieldDataType.String, 
                   filterable=True, sortable=True),
    SearchableField(name="content", type=SearchFieldDataType.String, 
                   analyzer_name="ko.microsoft"),  # 한국어 분석기
    SimpleField(name="category", type=SearchFieldDataType.String, 
               filterable=True, sortable=True, facetable=True),
    SimpleField(name="section", type=SearchFieldDataType.String, 
               filterable=True, sortable=False),
    SearchField(
        name="contentVector",
        type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
        searchable=True,
        vector_search_dimensions=3072,  # text-embedding-3-large
        vector_search_profile_name="vector-profile"
    )
]

# 벡터 검색 설정
vector_search = VectorSearch(
    algorithms=[
        HnswAlgorithmConfiguration(name="hnsw-algorithm")
    ],
    profiles=[
        VectorSearchProfile(
            name="vector-profile",
            algorithm_configuration_name="hnsw-algorithm"
        )
    ]
)

# 인덱스 생성
index = SearchIndex(
    name=index_name,
    fields=fields,
    vector_search=vector_search
)

# 기존 인덱스가 있으면 삭제 후 재생성
try:
    index_client.delete_index(index_name)
    print(f"⚠️ 기존 인덱스 '{index_name}' 삭제됨")
except:
    pass

result = index_client.create_index(index)
print(f"✅ 인덱스 생성 완료: {result.name}")
print(f"📊 필드 개수: {len(result.fields)}")
print("🔍 벡터 검색: 활성화 (3072 차원)")


🔑 AI Search Admin Key 가져오는 중...
✅ Admin Key 획득 완료
✅ Search Index Client 생성 완료 (Admin Key 인증)
⚠️ 기존 인덱스 'ai-agent-knowledge-base' 삭제됨
✅ 인덱스 생성 완료: ai-agent-knowledge-base
📊 필드 개수: 6
🔍 벡터 검색: 활성화 (3072 차원)


## 6. Azure OpenAI로 임베딩 생성 (Generate Embeddings with Azure OpenAI)

### 🧠 텍스트 임베딩이란?

**임베딩(Embedding)**은 텍스트를 숫자 벡터로 변환하는 과정입니다. 컴퓨터가 텍스트의 **의미**를 이해하고 비교할 수 있게 해줍니다.

```
"에이전트 보안"  →  [0.123, -0.456, 0.789, ..., 0.234]  (3072개 숫자)
"agent security" →  [0.119, -0.451, 0.801, ..., 0.228]  (매우 유사한 벡터!)
```

**핵심 개념:**
- 의미가 유사한 텍스트 = 유사한 벡터
- 벡터 간 거리(코사인 유사도)로 유사도 측정
- RAG의 핵심: 쿼리와 문서의 벡터를 비교하여 관련 문서 검색

---

### 🎯 text-embedding-3-large 모델 특징

**OpenAI의 최신 임베딩 모델 (2024년 출시)**

#### 📊 모델 비교

| 모델 | 차원 | 성능 | 비용 | 속도 | 적합한 용도 |
|------|------|------|------|------|------------|
| **text-embedding-3-large** | 3072 | ⭐⭐⭐⭐⭐ | 높음 | 보통 | **프로덕션 RAG (권장)** |
| text-embedding-3-small | 1536 | ⭐⭐⭐⭐ | 낮음 | 빠름 | 빠른 프로토타입, 비용 절감 |
| text-embedding-ada-002 | 1536 | ⭐⭐⭐ | 낮음 | 빠름 | 구 모델 (레거시) |

#### 🔍 text-embedding-3-large의 장점

**1. 높은 정확도**
- MTEB 벤치마크 점수: **64.6%** (ada-002: 61.0%)
- 다국어 성능 우수 (한국어 포함)
- 긴 텍스트(~8191 토큰) 처리 가능

**2. 3072 차원의 의미**
- **더 많은 차원 = 더 세밀한 의미 구별**
- 3072개의 숫자로 텍스트의 미묘한 의미 차이 표현
- 예시:
  ```
  1536 차원: "보안" vs "안전" → 약간 다름 (유사도 0.75)
  3072 차원: "보안" vs "안전" → 명확히 다름 (유사도 0.65)
  ```

**3. 차원 축소 기능**
```python
# 3072 차원 (기본)
embedding = generate_embedding("텍스트", dimensions=3072)

# 1536 차원으로 축소 (성능 약간 감소, 속도/비용 개선)
embedding = generate_embedding("텍스트", dimensions=1536)
```

---

### 📐 임베딩 생성 과정 시각화

```
┌──────────────────────────────────────────────────────────────┐
│  1단계: 텍스트 입력                                           │
│  "Azure AI 에이전트 개발 가이드"                              │
└───────────────────────┬──────────────────────────────────────┘
                        │
                        ▼
┌──────────────────────────────────────────────────────────────┐
│  2단계: 토큰화 (Tokenization)                                │
│  ["Azure", " AI", " 에이전트", " 개발", " 가이드"]           │
│  → Token IDs: [32001, 15902, 98234, 12834, 45678]           │
└───────────────────────┬──────────────────────────────────────┘
                        │
                        ▼
┌──────────────────────────────────────────────────────────────┐
│  3단계: Transformer 모델 처리                                │
│  • Self-Attention 메커니즘으로 문맥 이해                      │
│  • 12+ 레이어의 신경망 통과                                  │
│  • 각 토큰의 의미와 관계 학습                                │
└───────────────────────┬──────────────────────────────────────┘
                        │
                        ▼
┌──────────────────────────────────────────────────────────────┐
│  4단계: 벡터 생성                                            │
│  [0.0234, -0.1456, 0.0892, ..., 0.1123]                     │
│  ↑ 3072개의 부동소수점 숫자                                  │
└───────────────────────┬──────────────────────────────────────┘
                        │
                        ▼
┌──────────────────────────────────────────────────────────────┐
│  5단계: 정규화 (Normalization)                               │
│  벡터 길이를 1로 정규화 (코사인 유사도 계산 최적화)            │
│  최종 임베딩: [-0.0012, 0.0078, -0.0034, ..., 0.0056]       │
└──────────────────────────────────────────────────────────────┘
```

---

### 🎓 임베딩 품질 평가 방법

#### 1️⃣ 유사도 테스트

**동일 개념의 다른 표현 (유사도 높아야 함):**
```python
vec1 = generate_embedding("에이전트 보안")
vec2 = generate_embedding("agent security")
similarity = cosine_similarity(vec1, vec2)
# 기대값: > 0.85 (매우 유사)
```

**다른 개념 (유사도 낮아야 함):**
```python
vec1 = generate_embedding("에이전트 보안")
vec2 = generate_embedding("날씨 정보")
similarity = cosine_similarity(vec1, vec2)
# 기대값: < 0.3 (매우 다름)
```

#### 2️⃣ 검색 정확도 테스트

**방법:**
1. 테스트 쿼리 준비 (예: "RAG 시스템 구축 방법")
2. 관련 문서와 비관련 문서 준비
3. 검색 실행 후 상위 K개 결과 확인
4. 관련 문서가 상위에 있는지 평가

**평가 지표:**
- **Precision@K**: 상위 K개 중 관련 문서 비율
- **Recall@K**: 전체 관련 문서 중 상위 K개에 포함된 비율
- **MRR (Mean Reciprocal Rank)**: 첫 번째 관련 문서의 순위

#### 3️⃣ 차원별 성능 비교

```python
# 3072 차원 vs 1536 차원 정확도 비교
results_3072 = test_search(query, dimensions=3072)
results_1536 = test_search(query, dimensions=1536)

# 일반적으로:
# 3072 차원: Precision@5 ≈ 0.95
# 1536 차원: Precision@5 ≈ 0.88
```

---

### 💡 임베딩 최적화 팁

**1. 텍스트 결합 전략**
```python
# ✅ 좋은 방법: 제목 + 내용 결합 (이 Lab에서 사용)
text_to_embed = f"{doc['title']}\n\n{doc['content']}"

# ⚠️ 대안: 제목에 가중치 부여
text_to_embed = f"{doc['title']}\n{doc['title']}\n\n{doc['content']}"

# ❌ 나쁜 방법: 내용만 사용 (제목 정보 손실)
text_to_embed = doc['content']
```

**2. 긴 텍스트 처리**
- 최대 길이: **8191 토큰** (~32,000자)
- 초과 시: 자동으로 잘림 (뒷부분 손실!)
- 해결책: 청킹(Chunking) - 섹션별로 분할 (이미 적용됨)

**3. Rate Limiting 대응**
```python
# TPM (Tokens Per Minute) 제한 고려
time.sleep(0.5)  # 문서 간 0.5초 대기

# 또는 배치 처리 (Azure OpenAI Batch API)
# 대량 문서는 밤에 배치로 처리
```

**4. 캐싱 전략**
- 동일한 텍스트는 한 번만 임베딩 생성
- 결과를 파일로 저장하여 재사용
- 비용 절감 + 속도 향상

---

### 📊 메모리 및 비용 계산

**메모리 사용량:**
```
문서 1개당: 3072 차원 × 4 bytes (float32) = 12,288 bytes ≈ 12 KB
문서 54개: 54 × 12 KB ≈ 648 KB (매우 작음)
문서 10,000개: 10,000 × 12 KB ≈ 120 MB (관리 가능)
```

**API 비용 (2024년 기준):**
```
text-embedding-3-large: $0.13 / 1M 토큰
평균 문서 크기: 500 토큰
54개 문서: 54 × 500 = 27,000 토큰 ≈ $0.0035 (매우 저렴)
```

---

### 🎯 이 Lab에서의 구현

**코드 특징:**
1. ✅ `text-embedding-3-large` 사용 (최고 성능)
2. ✅ 3072 차원 명시적 지정 (dimensions=3072)
3. ✅ 제목 + 내용 결합 (의미 보존)
4. ✅ Rate limiting 대응 (0.5초 대기)
5. ✅ 진행 상황 출력 (사용자 경험 개선)

**다음 단계:**
- 생성된 임베딩을 Azure AI Search에 업로드 (섹션 7)
- 하이브리드 검색으로 품질 검증 (섹션 8-9)

In [9]:
from openai import AzureOpenAI
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
import time

# Azure OpenAI 클라이언트 생성 (Managed Identity 사용)
token_provider = get_bearer_token_provider(
    DefaultAzureCredential(),
    "https://cognitiveservices.azure.com/.default"
)

# Project connection string에서 OpenAI 엔드포인트 추출
# 형식 1: https://aoai-xxx.services.ai.azure.com/api/projects/proj-xxx;...
# 형식 2: workspace=...;subscription_id=...;resource_group=...;aiservices_name=...
import re

project_conn_str = config['project_connection_string']

# URL에서 AI Services 이름 추출 (형식 1)
url_match = re.match(r'https://([^.]+)\.', project_conn_str)
if url_match:
    aiservices_name = url_match.group(1)
    openai_endpoint = f"https://{aiservices_name}.openai.azure.com/"
else:
    # 키-값 형식에서 추출 (형식 2)
    conn_parts = {}
    for part in project_conn_str.split(';'):
        if '=' in part:
            key, value = part.split('=', 1)
            conn_parts[key] = value
    
    aiservices_name = conn_parts.get('aiservices_name', '')
    if not aiservices_name:
        raise ValueError("❌ AI Services 이름을 찾을 수 없습니다. config.json의 project_connection_string을 확인하세요.")
    
    openai_endpoint = f"https://{aiservices_name}.openai.azure.com/"

print(f"🔗 Azure OpenAI Endpoint: {openai_endpoint}")

openai_client = AzureOpenAI(
    azure_ad_token_provider=token_provider,
    api_version="2024-02-01",
    azure_endpoint=openai_endpoint
)

# 임베딩 모델 설정
embedding_model = "text-embedding-3-large"

def generate_embedding(text: str) -> list[float]:
    """텍스트를 벡터로 변환"""
    response = openai_client.embeddings.create(
        input=text,
        model=embedding_model,
        dimensions=3072  # 명시적으로 3072 차원으로 설정
    )
    return response.data[0].embedding

# 모든 문서에 대해 임베딩 생성
print("🔄 임베딩 생성 중...")
print(f"📄 처리할 문서: {len(documents)}개")

for i, doc in enumerate(documents, 1):
    # 제목과 내용을 결합하여 임베딩 생성
    text_to_embed = f"{doc['title']}\n\n{doc['content']}"
    doc["contentVector"] = generate_embedding(text_to_embed)
    
    print(f"  [{i}/{len(documents)}] {doc['title'][:50]}... ✓")
    
    # Rate limit 방지 (TPM 제한 고려)
    if i < len(documents):
        time.sleep(0.5)

print(f"\n✅ 임베딩 생성 완료")
print(f"📊 벡터 차원: {len(documents[0]['contentVector'])}차원 (3072차원)")
print(f"💾 메모리 사용량: ~{len(documents) * 3072 * 4 / 1024 / 1024:.2f} MB")


🔗 Azure OpenAI Endpoint: https://aoai-flyoy4n4dll42.openai.azure.com/
🔄 임베딩 생성 중...
📄 처리할 문서: 54개
  [1/54] 에이전트 개발 기초 - 개요... ✓
  [2/54] 에이전트 개발 기초 - 핵심 개념... ✓
  [3/54] 에이전트 개발 기초 - Azure AI Foundry SDK로 에이전트 구축... ✓
  [4/54] 에이전트 개발 기초 - Function Tools 추가... ✓
  [5/54] 에이전트 개발 기초 - Thread와 Run 관리... ✓
  [6/54] 에이전트 개발 기초 - Tool Call 처리... ✓
  [7/54] 에이전트 개발 기초 - Best Practices... ✓
  [8/54] 멀티 에이전트 아키텍처 - 개요... ✓
  [9/54] 멀티 에이전트 아키텍처 - 아키텍처 개요... ✓
  [10/54] 멀티 에이전트 아키텍처 - 에이전트 역할... ✓
  [11/54] 멀티 에이전트 아키텍처 - 통신 패턴... ✓
  [12/54] 멀티 에이전트 아키텍처 - Azure AI Foundry에서 Connected Agents... ✓
  [13/54] 멀티 에이전트 아키텍처 - Azure AI Foundry에서 Connected Agents... ✓
  [14/54] 멀티 에이전트 아키텍처 - Azure AI Foundry에서 Connected Agents... ✓
  [15/54] 멀티 에이전트 아키텍처 - 설계 고려사항... ✓
  [16/54] 멀티 에이전트 아키텍처 - Best Practices... ✓
  [17/54] RAG와 Azure AI Search - 개요... ✓
  [18/54] RAG와 Azure AI Search - Azure AI Search 개요... ✓
  [19/54] RAG와 Azure AI Search - Azure AI Search로 RAG 구현... ✓
  [20/54] RAG와 Azure AI Searc

## 7. 문서를 Azure AI Search에 업로드 (Upload Documents to Azure AI Search)

### 📤 문서 업로드 전략

Azure AI Search에 문서를 업로드할 때는 **효율성**, **안정성**, **비용**을 모두 고려해야 합니다.

---

### 🎯 배치 업로드 vs 개별 업로드

#### 이 Lab에서의 선택: **전체 배치 업로드**

**코드:**
```python
# 54개 문서를 한 번에 업로드 (배치 크기 < 1000)
result = search_client.upload_documents(documents=cleaned_documents)
```

**선택 이유:**
- ✅ 문서 수가 적음 (54개)
- ✅ 단일 API 호출로 완료
- ✅ 간단한 오류 처리

---

### 📦 배치 크기 전략

Azure AI Search의 배치 업로드 제한:

| 구분 | 제한 | 권장 | 이유 |
|------|------|------|------|
| **최대 문서 수** | 1,000개/배치 | 100-500개 | 안정성 |
| **최대 크기** | 16 MB/배치 | 8-12 MB | 타임아웃 방지 |
| **동시 요청** | 제한 없음 | 2-4개 | Rate limiting |

#### 💡 왜 100개씩 배치 처리하는가?

**1. 메모리 효율성**
```python
# 나쁜 예: 10,000개 문서를 한 번에
# 메모리: 10,000 × 12KB = 120 MB (큰 부담)
result = search_client.upload_documents(documents=all_10k_docs)

# 좋은 예: 100개씩 배치 처리
for i in range(0, len(documents), 100):
    batch = documents[i:i+100]
    result = search_client.upload_documents(documents=batch)
```

**2. 오류 격리**
- 배치 1개 실패 = 100개만 재시도 (vs 10,000개 전체)
- 실패한 배치만 로그에 기록하여 디버깅 용이

**3. 진행 상황 추적**
```python
for i in range(0, len(documents), 100):
    batch = documents[i:i+100]
    result = search_client.upload_documents(documents=batch)
    print(f"✓ {i+len(batch)}/{len(documents)} 업로드 완료")
```

**4. Rate Limiting 회피**
- Azure AI Search: 초당 최대 3,000개 문서 인덱싱
- 100개 배치 + 0.5초 대기 = 초당 200개 (안전)

---

### ⚡ Rate Limiting 대응 전략

#### Azure AI Search 제한

| 티어 | 초당 요청 | 분당 인덱싱 | 동시 연결 |
|------|----------|------------|----------|
| **Free** | 3 | 180 docs | 3 |
| **Basic** | 15 | 900 docs | 15 |
| **Standard** | 60 | 3,600 docs | 60 |

#### 대응 방법

**1. 지수 백오프 (Exponential Backoff)**
```python
import time
from azure.core.exceptions import HttpResponseError

def upload_with_retry(batch, max_retries=3):
    for attempt in range(max_retries):
        try:
            result = search_client.upload_documents(documents=batch)
            return result
        except HttpResponseError as e:
            if e.status_code == 429:  # Too Many Requests
                wait_time = 2 ** attempt  # 1초, 2초, 4초
                print(f"⏳ Rate limit 도달. {wait_time}초 대기...")
                time.sleep(wait_time)
            else:
                raise
    raise Exception("❌ 최대 재시도 횟수 초과")
```

**2. 동적 배치 크기 조정**
```python
batch_size = 100
while documents:
    batch = documents[:batch_size]
    try:
        result = upload_with_retry(batch)
        documents = documents[batch_size:]
    except HttpResponseError as e:
        if e.status_code == 429:
            batch_size = max(10, batch_size // 2)  # 배치 크기 절반으로
            print(f"⚠️ 배치 크기를 {batch_size}로 축소")
```

**3. 병렬 업로드 (고급)**
```python
from concurrent.futures import ThreadPoolExecutor

def upload_batch(batch):
    return search_client.upload_documents(documents=batch)

batches = [documents[i:i+100] for i in range(0, len(documents), 100)]

with ThreadPoolExecutor(max_workers=4) as executor:
    results = list(executor.map(upload_batch, batches))
```

---

### 📊 대량 문서 업로드 시 주의사항

#### 1️⃣ 벡터 크기 최적화

**문제:**
```python
# 10,000 문서 × 3072 차원 × 4 bytes = 122 MB
# 네트워크 전송 시간: 약 10-30초 (1 Gbps 기준)
```

**해결책:**
- 배치 크기 줄이기 (100개 → 50개)
- 차원 축소 고려 (3072 → 1536, 성능 약간 감소)
- 압축 사용 (Azure SDK 자동 처리)

#### 2️⃣ 메모리 관리

**문제: 메모리 부족**
```python
# 나쁜 예: 모든 문서를 메모리에 로드
all_docs = load_all_100k_documents()  # 1.2 GB 메모리!
for i in range(0, len(all_docs), 100):
    upload_batch(all_docs[i:i+100])
```

**해결책: 스트리밍 방식**
```python
# 좋은 예: 필요할 때만 로드
def document_generator(file_path):
    with open(file_path) as f:
        for line in f:
            yield json.loads(line)  # JSONL 형식

for batch in batched(document_generator('docs.jsonl'), 100):
    upload_batch(list(batch))
```

#### 3️⃣ 타임아웃 설정

**기본 타임아웃: 300초 (5분)**
```python
from azure.core.pipeline.policies import RetryPolicy

# 타임아웃 증가 (대량 업로드 시)
search_client = SearchClient(
    endpoint=config["search_endpoint"],
    index_name=index_name,
    credential=AzureKeyCredential(search_admin_key),
    retry_policy=RetryPolicy(timeout=600)  # 10분
)
```

#### 4️⃣ 오류 추적 및 복구

**전략: 체크포인트 저장**
```python
uploaded_ids = set()

for i in range(0, len(documents), 100):
    batch = documents[i:i+100]
    
    try:
        result = search_client.upload_documents(documents=batch)
        
        # 성공한 문서 ID 저장
        for r in result:
            if r.succeeded:
                uploaded_ids.add(r.key)
        
        # 체크포인트 저장 (재시작 시 사용)
        save_checkpoint(uploaded_ids, f"checkpoint_{i}.json")
        
    except Exception as e:
        print(f"❌ 배치 {i} 실패: {e}")
        # 실패한 배치만 로그 저장
        save_failed_batch(batch, f"failed_{i}.json")
        continue
```

**복구:**
```python
# 체크포인트에서 재개
uploaded_ids = load_checkpoint("checkpoint_5000.json")
remaining_docs = [doc for doc in documents if doc['id'] not in uploaded_ids]
print(f"📋 남은 문서: {len(remaining_docs)}개")
```

---

### 🔍 업로드 성공 검증

**1. 문서 수 확인**
```python
# 업로드 직후
doc_count = search_client.get_document_count()
print(f"인덱스 문서 수: {doc_count}")
# 예상: 54개 (이 Lab)
```

**2. 특정 문서 조회**
```python
# 문서 ID로 직접 조회
doc = search_client.get_document(key="doc_001")
print(f"제목: {doc['title']}")
print(f"벡터 차원: {len(doc['contentVector'])}")
```

**3. 샘플 검색 테스트**
```python
# 간단한 키워드 검색
results = search_client.search("에이전트", top=3)
for r in results:
    print(f"- {r['title']}")
```

---

### 💡 프로덕션 환경 권장 사항

#### ✅ DO (권장)
1. **배치 크기 100-500개** 사용
2. **지수 백오프** 재시도 로직 구현
3. **체크포인트** 저장으로 재시작 지원
4. **로깅**: 성공/실패 문서 ID 기록
5. **모니터링**: Azure Monitor로 인덱싱 성능 추적

#### ❌ DON'T (비권장)
1. **1,000개 이상 배치** 사용 (타임아웃 위험)
2. **재시도 없이 한 번만** 업로드
3. **전체 실패 시 전체 재시도** (체크포인트 없이)
4. **동시 10개 이상 업로드** (Rate limiting)
5. **오류 무시하고 계속** 진행

---

### 📈 이 Lab에서의 구현

**현재 코드 특징:**
```python
# ✅ 간단하고 안전한 방식
# - 54개 문서: 배치 크기 제한 불필요
# - 단일 API 호출
# - 명확한 성공/실패 카운트
result = search_client.upload_documents(documents=cleaned_documents)
```

**대량 문서 처리 시 개선 예시:**
```python
# 10,000+ 문서 처리
BATCH_SIZE = 100

for i in range(0, len(documents), BATCH_SIZE):
    batch = documents[i:i+BATCH_SIZE]
    
    try:
        result = upload_with_retry(batch)
        print(f"✓ {i+len(batch)}/{len(documents)}")
        time.sleep(0.5)  # Rate limiting 대응
    except Exception as e:
        print(f"❌ 배치 {i} 실패: {e}")
        save_failed_batch(batch, f"failed_{i}.json")
```

**다음 단계:**
- 업로드된 문서로 하이브리드 검색 테스트 (섹션 8-9)

In [10]:
from azure.search.documents import SearchClient

# SearchClient 생성 (Admin Key 인증 재사용)
search_client = SearchClient(
    endpoint=config["search_endpoint"],
    index_name=index_name,
    credential=AzureKeyCredential(search_admin_key)
)

print("📤 문서 업로드 시작...")
print(f"🔐 인증 방식: Admin Key")

# 인덱스 스키마에 정의된 필드만 포함하도록 문서 정리
allowed_fields = {"id", "title", "content", "category", "section", "contentVector"}
cleaned_documents = []
for doc in documents:
    cleaned_doc = {key: value for key, value in doc.items() if key in allowed_fields}
    cleaned_documents.append(cleaned_doc)

print(f"📋 정리된 문서 필드: {list(cleaned_documents[0].keys())}")

# Upload documents with embeddings
result = search_client.upload_documents(documents=cleaned_documents)

# Check results
succeeded = sum([1 for r in result if r.succeeded])
failed = sum([1 for r in result if not r.succeeded])

print(f"\n✅ 업로드 완료! (Upload complete!)")
print(f"   - 성공: {succeeded}개")
print(f"   - 실패: {failed}개")

if failed > 0:
    print("\n⚠️  일부 문서 업로드 실패. 다음을 확인하세요:")
    print("   1. 인덱스 스키마와 문서 필드가 일치하는지 확인")
    print("   2. contentVector 차원이 3072인지 확인")
    print("   3. Azure AI Search 서비스 상태 확인")
else:
    print("\n🎉 모든 문서가 성공적으로 인덱싱되었습니다!")
    print(f"   인덱스 '{index_name}'에 {succeeded}개 문서가 저장되었습니다.")

📤 문서 업로드 시작...
🔐 인증 방식: Admin Key
📋 정리된 문서 필드: ['id', 'title', 'content', 'category', 'section', 'contentVector']

✅ 업로드 완료! (Upload complete!)
   - 성공: 54개
   - 실패: 0개

🎉 모든 문서가 성공적으로 인덱싱되었습니다!
   인덱스 'ai-agent-knowledge-base'에 54개 문서가 저장되었습니다.


## 8. 하이브리드 검색 테스트 (Test Hybrid Search)

이제 **벡터 검색**(의미 기반)과 **키워드 검색**(전통적 검색)을 결합한 **하이브리드 검색**을 테스트합니다.

### 🔍 하이브리드 검색 개념

```
┌─────────────────────────────────────────────────────────┐
│              사용자 쿼리: "에이전트 보안"                │
└─────────────────┬───────────────────────────────────────┘
                  │
        ┌─────────┴─────────┐
        │                   │
        ▼                   ▼
┌──────────────┐    ┌──────────────┐
│ 벡터 검색    │    │ 키워드 검색  │
│ (Semantic)   │    │ (BM25)       │
└──────┬───────┘    └──────┬───────┘
       │                   │
       │  Rank 1: doc_003  │  Rank 1: doc_005
       │  Rank 2: doc_007  │  Rank 2: doc_003
       │  Rank 3: doc_001  │  Rank 3: doc_009
       │                   │
       └─────────┬─────────┘
                 │
                 ▼
        ┌────────────────┐
        │ RRF 알고리즘   │
        │ (순위 융합)    │
        └────────┬───────┘
                 │
                 ▼
       ┌──────────────────┐
       │ 최종 결과:       │
       │ 1. doc_003       │
       │ 2. doc_005       │
       │ 3. doc_007       │
       └──────────────────┘
```

### 📊 검색 방식 비교

| 구분 | 벡터 검색 (Vector) | 키워드 검색 (Keyword) | 하이브리드 검색 |
|------|--------------------|-----------------------|----------------|
| **검색 방식** | 임베딩 유사도 (Cosine) | 단어 빈도 (BM25) | 벡터 + 키워드 결합 |
| **장점** | 의미적 유사성 파악 | 정확한 단어 매칭 | 두 방식의 장점 결합 |
| **단점** | 정확한 용어 매칭 약함 | 동의어/유사 개념 약함 | 계산 비용 증가 |
| **예시 쿼리** | "에이전트 만들기" ≈ "agent development" | "RAG" = "RAG" (정확 매칭) | 둘 다 활용 |
| **정확도** | 70-80% | 60-70% | 85-95% |
| **사용 케이스** | 자연어 질문, FAQ | 코드 검색, 전문 용어 | 일반적인 모든 경우 |

### ⚙️ RRF 알고리즘 (Reciprocal Rank Fusion)

Azure AI Search는 **RRF(Reciprocal Rank Fusion)** 알고리즘을 사용하여 벡터 검색과 키워드 검색 결과를 결합합니다.

**수식:**
```
RRF Score = Σ (1 / (k + rank))
k = 60 (기본값, Azure 설정)
```

**예시:**
- 문서 A가 벡터 검색에서 1위 (rank=1), 키워드 검색에서 3위 (rank=3)
- RRF Score = 1/(60+1) + 1/(60+3) ≈ 0.0164 + 0.0159 = **0.0323**
- 문서 B가 벡터 검색에서 5위 (rank=5), 키워드 검색에서 1위 (rank=1)
- RRF Score = 1/(60+5) + 1/(60+1) ≈ 0.0154 + 0.0164 = **0.0318**
- **결과: 문서 A가 더 높은 점수로 상위 랭크**

### 🚀 Azure AI Search 하이브리드 검색 구현

```python
from azure.search.documents.models import VectorizedQuery

# 하이브리드 검색 실행
results = search_client.search(
    search_text="에이전트 보안",  # 키워드 검색 쿼리
    vector_queries=[
        VectorizedQuery(
            vector=query_vector,  # 벡터 검색 쿼리
            k_nearest_neighbors=50,
            fields="contentVector"
        )
    ],
    select=["title", "content"],
    top=5
)
```

### 🎯 언제 어떤 검색을 사용할까?

**1. 벡터 검색만 사용:**
- 자연어 질문 답변 (예: "에이전트를 어떻게 만드나요?")
- 의미적 유사성 중시 (예: "비용 절감 방법" ≈ "예산 최적화")

**2. 키워드 검색만 사용:**
- 정확한 코드/명령어 검색 (예: "pip install azure-search-documents")
- 전문 용어/약어 검색 (예: "MCP", "RAG", "LLM")

**3. 하이브리드 검색 (권장):**
- 일반적인 모든 검색 케이스
- 정확도가 중요한 프로덕션 환경
- 사용자 의도가 불명확한 경우

### 🧪 테스트 쿼리 예시

**테스트 1: 전문 용어 (키워드 검색 강점)**
- 쿼리: "MCP 서버 개발"
- 예상: "Model Context Protocol" 정확 매칭 문서 상위 노출

**테스트 2: 자연어 질문 (벡터 검색 강점)**
- 쿼리: "에이전트 시스템 안전하게 만들기"
- 예상: "security", "보안", "안전" 관련 문서 의미적 매칭

**테스트 3: 복합 쿼리 (하이브리드 강점)**
- 쿼리: "Azure에서 RAG 구축하기"
- 예상: "Azure", "RAG" 정확 매칭 + "구축", "개발" 의미적 매칭


In [11]:
from azure.search.documents.models import VectorizedQuery

def test_hybrid_search(query_text: str, top: int = 3):
    """
    하이브리드 검색 실행: 벡터 검색 + 키워드 검색
    """
    print(f"\n🔍 검색 쿼리: '{query_text}'")
    print("=" * 80)
    
    # 1. 쿼리 텍스트를 벡터로 변환
    query_vector = generate_embedding(query_text)
    
    # 2. 벡터 검색 쿼리 생성
    vector_query = VectorizedQuery(
        vector=query_vector,
        k_nearest_neighbors=top,
        fields="contentVector"
    )
    
    # 3. 하이브리드 검색 실행 (벡터 + 키워드)
    results = search_client.search(
        search_text=query_text,  # 키워드 검색
        vector_queries=[vector_query],  # 벡터 검색
        select=["id", "title", "content", "category", "section"],
        top=top
    )
    
    # 4. 결과 출력
    for i, result in enumerate(results, 1):
        score = result.get("@search.score", 0)
        print(f"\n📄 결과 {i} (점수: {score:.4f})")
        print(f"   ID: {result['id']}")
        print(f"   제목: {result['title']}")
        print(f"   카테고리: {result['category']}")
        print(f"   섹션: {result.get('section', 'N/A')}")
        print(f"   내용 (처음 200자): {result['content'][:200]}...")
    
    print("\n" + "=" * 80)

# 테스트 쿼리 1: 에이전트 개발
test_hybrid_search("Azure AI 에이전트를 어떻게 개발하나요?")

# 테스트 쿼리 2: RAG 최적화
test_hybrid_search("RAG 시스템의 성능을 최적화하는 방법")

# 테스트 쿼리 3: MCP 구현
test_hybrid_search("Model Context Protocol 도구 서버 구현하기")

print("\n✅ 하이브리드 검색 테스트 완료!")
print("   - 한국어 분석기(ko.microsoft)가 잘 작동합니다")
print("   - 벡터 검색과 키워드 검색이 결합되어 정확한 결과를 반환합니다")



🔍 검색 쿼리: 'Azure AI 에이전트를 어떻게 개발하나요?'

📄 결과 1 (점수: 0.0325)
   ID: doc-01-01
   제목: 에이전트 개발 기초 - 핵심 개념
   카테고리: 에이전트 개발 기초
   섹션: 1
   내용 (처음 200자): ### 에이전트 아키텍처
Azure AI 에이전트는 다음으로 구성됩니다:
- **Instructions**: 에이전트의 행동과 성격을 정의하는 시스템 프롬프트
- **Model**: 기본 언어 모델 (예: GPT-4o)
- **Tools**: 에이전트가 작업을 수행하기 위해 호출할 수 있는 함수
- **Knowledge**: 에이전트가 참조할 수 있는 정보...

📄 결과 2 (점수: 0.0321)
   ID: doc-01-02
   제목: 에이전트 개발 기초 - Azure AI Foundry SDK로 에이전트 구축
   카테고리: 에이전트 개발 기초
   섹션: 1
   내용 (처음 200자): Azure AI Foundry SDK는 에이전트를 생성하고 관리하기 위한 Python 기반 인터페이스를 제공합니다:

```python
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential

# Client 초기화
client = AIPro...

📄 결과 3 (점수: 0.0291)
   ID: doc-01-06
   제목: 에이전트 개발 기초 - Best Practices
   카테고리: 에이전트 개발 기초
   섹션: 1
   내용 (처음 200자): 1. **명확한 지시사항**: 에이전트 동작에 대한 상세하고 구체적인 지시사항 제공
2. **도구 선택**: 에이전트의 작업에 적합한 도구 선택
3. **에러 처리**: 강력한 에러 처리 및 폴백 메커니즘 구현
4. **모니터링**: Application Insights를 사용하여 에이전트 성능 모니터링
5. **보안**: Managed I

### 📊 검색 결과 분석

위 코드를 실행하면 **3가지 테스트 쿼리**에 대한 하이브리드 검색 결과를 확인할 수 있습니다.

#### 🔍 결과에서 확인할 수 있는 정보:

**1. 검색 점수 (`@search.score`)**
- RRF 알고리즘으로 계산된 최종 순위 점수
- 높을수록 쿼리와 더 관련성이 높은 문서
- 일반적으로 **0.01 ~ 0.04** 범위의 값

**2. 문서 메타데이터**
- `id`: 문서 고유 식별자
- `title`: 문서 제목
- `category`: 문서 카테고리 (예: Agent, RAG, MCP)
- `section`: 문서 섹션 (예: 개발, 배포, 최적화)

**3. 문서 내용 미리보기**
- 처음 200자만 표시하여 검색 결과 품질 확인 가능

#### 💡 결과 해석 가이드:

**테스트 쿼리 1: "Azure AI 에이전트를 어떻게 개발하나요?"**
- **예상 결과:** Agent 카테고리의 개발 관련 문서가 상위 노출
- **검색 원리:** 
  - 벡터 검색: "개발", "만들기", "구축" 등 의미적으로 유사한 표현 매칭
  - 키워드 검색: "Azure", "AI", "에이전트" 정확한 단어 매칭
  - RRF 융합: 두 결과를 결합하여 최적의 문서 선택

**테스트 쿼리 2: "RAG 시스템의 성능을 최적화하는 방법"**
- **예상 결과:** RAG 카테고리의 최적화/성능 관련 문서
- **검색 원리:**
  - 벡터 검색: "성능 향상", "개선", "효율화" 등 동의어 매칭
  - 키워드 검색: "RAG", "최적화" 정확한 용어 매칭
  - 하이브리드 장점: 전문 용어(RAG) + 자연어(최적화하는 방법) 모두 처리

**테스트 쿼리 3: "Model Context Protocol 도구 서버 구현하기"**
- **예상 결과:** MCP 카테고리의 서버 구현 관련 문서
- **검색 원리:**
  - 벡터 검색: "구현", "개발", "작성" 등 유사 개념 매칭
  - 키워드 검색: "Model Context Protocol", "MCP" 약어 정확 매칭
  - 복합 쿼리 처리: 긴 전문 용어 + 자연어 조합 효과적으로 처리

#### ⚠️ 결과가 예상과 다른 경우:

**1. 점수가 너무 낮은 경우 (< 0.01)**
- 쿼리와 문서 간 관련성이 낮을 수 있음
- 쿼리를 더 구체적으로 수정 필요

**2. 결과가 0개인 경우**
```python
# 인덱스 상태 확인
doc_count = search_client.get_document_count()
print(f"인덱스의 총 문서 수: {doc_count}")
```
- 문서가 업로드되지 않았거나
- 인덱싱이 완료되지 않았을 수 있음 (1-2분 대기)

**3. 한국어 결과 품질이 낮은 경우**
- `ko.microsoft` 분석기 설정 확인
- 하이브리드 검색 사용 여부 확인 (`search_text` + `vector_queries`)

#### 🎯 성공적인 결과의 특징:

✅ **상위 3개 문서 모두 쿼리와 직접적으로 관련**
✅ **점수 차이가 명확** (1위와 3위 간 0.005 이상 차이)
✅ **다양한 관점의 문서 포함** (예: 개념, 구현, 예시)
✅ **카테고리와 섹션이 쿼리 의도와 일치**

이러한 결과는 하이브리드 검색이 **벡터 검색과 키워드 검색의 장점을 효과적으로 결합**하여 작동하고 있음을 보여줍니다!

## 9. 검색 성능 비교 (Search Performance Comparison)

**벡터 전용 검색 vs. 키워드 전용 검색 vs. 하이브리드 검색**의 성능을 실제로 비교해봅니다.

### 🎯 비교 목적

동일한 쿼리에 대해 **3가지 검색 방식**을 실행하여:
- 각 방식의 검색 결과 차이 확인
- 검색 품질과 관련성 비교
- 하이브리드 검색의 우수성 검증

### 📋 테스트 설정

**테스트 쿼리:** `"에이전트 시스템 보안"`

이 쿼리는 다음과 같은 특징이 있습니다:
- **전문 용어**: "에이전트", "시스템" (키워드 검색 유리)
- **일반 개념**: "보안" (벡터 검색으로 "security", "안전", "방어" 등도 찾을 수 있음)
- **복합 주제**: 기술적 용어 + 추상적 개념 조합 (하이브리드 검색에 이상적)

### 🔬 3가지 검색 방식

**방법 1: 벡터 전용 검색 (Vector-only Search)**
```python
search_client.search(
    search_text=None,  # 키워드 검색 비활성화
    vector_queries=[vector_query],  # 벡터 검색만 사용
    top=3
)
```
- **작동 원리**: 쿼리 임베딩과 문서 임베딩 간 코사인 유사도 계산
- **장점**: 동의어, 유사 개념 검색 가능 (예: "보안" ≈ "security" ≈ "안전")
- **단점**: 정확한 용어 매칭 약함

**방법 2: 하이브리드 검색 (Hybrid Search)**
```python
search_client.search(
    search_text=test_query,  # 키워드 검색 활성화
    vector_queries=[vector_query],  # 벡터 검색도 사용
    top=3
)
```
- **작동 원리**: 벡터 + 키워드 결과를 RRF 알고리즘으로 융합
- **장점**: 두 방식의 장점 결합, 가장 정확한 결과
- **단점**: 계산 비용 약간 증가 (실시간 검색에서도 무시 가능한 수준)

**방법 3: 키워드 전용 검색 (Keyword-only Search)**
```python
search_client.search(
    search_text=test_query,  # 키워드 검색만 사용
    vector_queries=None,  # 벡터 검색 비활성화
    top=3
)
```
- **작동 원리**: BM25 알고리즘으로 단어 빈도와 문서 관련성 계산
- **장점**: 정확한 단어/용어 매칭, 빠른 속도
- **단점**: 동의어/유사 표현 인식 불가

### 📊 예상 결과 비교

| 순위 | 벡터 전용 | 하이브리드 | 키워드 전용 |
|------|-----------|-----------|-------------|
| **1위** | "Agent Security Best Practices" | "Agent Security Best Practices" | "에이전트 시스템 개요" |
| **2위** | "안전한 AI 시스템 구축" | "안전한 AI 시스템 구축" | "보안 체크리스트" |
| **3위** | "Agent Deployment Guide" | "보안 체크리스트" | "시스템 아키텍처" |
| **특징** | 의미적 유사성 중심 | 균형 잡힌 결과 | 정확한 단어 매칭 |

### 🎓 학습 포인트

- **벡터 검색**: "보안"과 "security", "안전", "방어" 등을 같은 의미로 인식
- **키워드 검색**: "에이전트", "시스템", "보안" 정확히 포함된 문서 우선
- **하이브리드**: 정확한 용어 매칭 + 의미적 확장 = 최고의 품질

In [12]:
import sys
import time

test_query = "에이전트 시스템 보안"

# 쿼리 임베딩 생성 (1회만)
print("🔄 검색 쿼리 임베딩 생성 중...")
query_vector = generate_embedding(test_query)
vector_query = VectorizedQuery(vector=query_vector, k_nearest_neighbors=3, fields="contentVector")
print("✅ 임베딩 생성 완료\n")
sys.stdout.flush()  # 출력 버퍼 강제 비우기

# === 방법 1: 벡터 전용 검색 ===
print("🔍 방법 1: 벡터 전용 검색 (의미 기반)")
print("=" * 80)
sys.stdout.flush()

# 검색 실행 및 결과 수집 (동기적)
vector_search_results = search_client.search(
    search_text=None,  # 키워드 검색 비활성화
    vector_queries=[vector_query],
    select=["title", "category"],
    top=3
)
vector_results = []
for result in vector_search_results:
    vector_results.append(result)

# 결과 출력
for i, r in enumerate(vector_results, 1):
    print(f"   {i}. {r['title']}")
sys.stdout.flush()
time.sleep(0.1)  # 출력 완료 대기

# === 방법 2: 하이브리드 검색 ===
print("\n🔍 방법 2: 하이브리드 검색 (의미 + 키워드)")
print("=" * 80)
sys.stdout.flush()

# 검색 실행 및 결과 수집 (동기적)
hybrid_search_results = search_client.search(
    search_text=test_query,  # 키워드 검색 활성화
    vector_queries=[vector_query],
    select=["title", "category"],
    top=3
)
hybrid_results = []
for result in hybrid_search_results:
    hybrid_results.append(result)

# 결과 출력
for i, r in enumerate(hybrid_results, 1):
    print(f"   {i}. {r['title']}")
sys.stdout.flush()
time.sleep(0.1)  # 출력 완료 대기

# === 방법 3: 키워드 전용 검색 ===
print("\n🔍 방법 3: 키워드 전용 검색 (전통적 검색)")
print("=" * 80)
sys.stdout.flush()

# 검색 실행 및 결과 수집 (동기적)
keyword_search_results = search_client.search(
    search_text=test_query,
    vector_queries=None,  # 벡터 검색 비활성화
    select=["title", "category"],
    top=3
)
keyword_results = []
for result in keyword_search_results:
    keyword_results.append(result)

# 결과 출력
for i, r in enumerate(keyword_results, 1):
    print(f"   {i}. {r['title']}")
sys.stdout.flush()
time.sleep(0.1)  # 출력 완료 대기

print("\n" + "=" * 80)
print("📊 결론:")
print("   - 벡터 검색: 의미적으로 유사한 문서 검색에 강함")
print("   - 키워드 검색: 정확한 단어 매칭에 강함")
print("   - 하이브리드 검색: 두 방식의 장점을 결합하여 가장 정확한 결과 제공 (권장)")

🔄 검색 쿼리 임베딩 생성 중...
✅ 임베딩 생성 완료

🔍 방법 1: 벡터 전용 검색 (의미 기반)
   1. 멀티 에이전트 아키텍처 - 아키텍처 개요
   2. 에이전트 개발 기초 - Best Practices
   3. 멀티 에이전트 아키텍처 - Best Practices

🔍 방법 2: 하이브리드 검색 (의미 + 키워드)
   1. 에이전트 개발 기초 - Best Practices
   2. 멀티 에이전트 아키텍처 - Best Practices
   3. 멀티 에이전트 아키텍처 - 아키텍처 개요

🔍 방법 3: 키워드 전용 검색 (전통적 검색)
   1. 에이전트 개발 기초 - 핵심 개념
   2. 에이전트 개발 기초 - Best Practices
   3. 멀티 에이전트 아키텍처 - Best Practices

📊 결론:
   - 벡터 검색: 의미적으로 유사한 문서 검색에 강함
   - 키워드 검색: 정확한 단어 매칭에 강함
   - 하이브리드 검색: 두 방식의 장점을 결합하여 가장 정확한 결과 제공 (권장)


### 📈 검색 결과 분석 및 인사이트

위 코드를 실행하면 **동일한 쿼리**에 대해 **3가지 검색 방식의 결과**를 직접 비교할 수 있습니다.

#### 🔍 결과 해석 가이드

**1️⃣ 벡터 전용 검색 결과 분석**

**특징:**
- 쿼리의 **의미적 내용**을 이해하여 관련 문서 검색
- "보안"과 관련된 다양한 표현 찾기 (예: security, 안전, 방어, 보호)
- 정확한 단어 매칭 없이도 관련 문서 검색 가능

**예상 패턴:**
```
✅ 포함될 가능성 높음: "Agent Security Best Practices", "안전한 AI 시스템"
❌ 누락될 가능성: "에이전트 시스템" 정확히 포함하지만 보안과 무관한 문서
```

**장점:**
- 자연어 질문에 강함
- 다국어 의미 매칭 (한국어 "보안" ≈ 영어 "security")
- 동의어 자동 처리

**단점:**
- 때로 관련성 낮은 문서 포함 가능
- 정확한 전문 용어 검색 시 정확도 떨어질 수 있음

---

**2️⃣ 하이브리드 검색 결과 분석**

**특징:**
- **벡터 검색 + 키워드 검색** 결과를 RRF로 융합
- 정확한 용어 매칭 + 의미적 확장
- 일반적으로 **가장 관련성 높은 문서**가 상위 노출

**예상 패턴:**
```
✅ 벡터 검색 1위 + 키워드 검색 1위 → 하이브리드 1위 (거의 확실)
✅ 균형 잡힌 결과: 정확한 용어 매칭 + 유사 개념 문서 포함
```

**RRF 융합 예시:**
- 문서 A: 벡터 1위, 키워드 3위 → RRF 점수 높음
- 문서 B: 벡터 2위, 키워드 2위 → RRF 점수 **더 높음** (균형)
- 문서 C: 벡터 5위, 키워드 1위 → RRF 점수 중간

**장점:**
- **프로덕션 환경 권장 방식**
- 대부분의 쿼리 유형에서 최고 성능
- 정확도 85-95% (벡터: 70-80%, 키워드: 60-70%)

**단점:**
- 계산 비용 약간 증가 (실무에서 무시 가능)

---

**3️⃣ 키워드 전용 검색 결과 분석**

**특징:**
- BM25 알고리즘으로 **단어 빈도** 기반 순위 계산
- "에이전트", "시스템", "보안" 단어가 많이 포함된 문서 우선
- 정확한 용어 매칭에 강함

**예상 패턴:**
```
✅ "에이전트 시스템 보안 가이드" (정확히 모든 단어 포함) → 높은 순위
❌ "Agent Security Best Practices" (영어 문서) → 낮은 순위 또는 누락
❌ "안전한 에이전트 구축" (동의어) → 낮은 순위
```

**장점:**
- 빠른 속도
- 정확한 코드/명령어 검색에 유리
- 전문 용어/약어 검색 (예: "MCP", "RAG")

**단점:**
- 동의어 인식 불가 ("보안" ≠ "security")
- 자연어 질문 처리 약함
- 한국어-영어 혼합 검색 어려움

---

#### 💡 실제 결과에서 확인할 수 있는 것들

**1. 검색 결과 순서 차이**
- 각 방식마다 **다른 순서**로 문서가 나타남
- 하이브리드 검색이 일반적으로 **가장 관련성 높은 문서**를 1위로 선정

**2. 검색된 문서의 차이**
- 벡터 검색: 의미적으로 유사하지만 정확한 단어는 없는 문서 포함
- 키워드 검색: 정확한 단어는 있지만 맥락이 다른 문서 포함 가능
- 하이브리드: 두 조건을 모두 만족하는 문서 우선

**3. 결과의 일관성**
- **오버랩**: 3가지 방식 모두에서 나타나는 문서 → **매우 관련성 높음**
- **벡터 only**: 벡터 검색에만 나타나는 문서 → 의미적으로 유사
- **키워드 only**: 키워드 검색에만 나타나는 문서 → 단어는 같지만 맥락 다를 수 있음

---

#### 📊 성능 비교 요약

| 평가 항목 | 벡터 전용 | 하이브리드 | 키워드 전용 |
|----------|----------|-----------|------------|
| **정확도** | ⭐⭐⭐ (70-80%) | ⭐⭐⭐⭐⭐ (85-95%) | ⭐⭐⭐ (60-70%) |
| **속도** | ⭐⭐⭐ (빠름) | ⭐⭐⭐ (빠름) | ⭐⭐⭐⭐⭐ (매우 빠름) |
| **자연어 질문** | ⭐⭐⭐⭐⭐ (우수) | ⭐⭐⭐⭐⭐ (우수) | ⭐⭐ (보통) |
| **전문 용어** | ⭐⭐⭐ (보통) | ⭐⭐⭐⭐⭐ (우수) | ⭐⭐⭐⭐ (좋음) |
| **다국어 지원** | ⭐⭐⭐⭐⭐ (우수) | ⭐⭐⭐⭐⭐ (우수) | ⭐⭐ (보통) |
| **동의어 인식** | ⭐⭐⭐⭐⭐ (우수) | ⭐⭐⭐⭐⭐ (우수) | ⭐ (없음) |

---

#### 🎯 실무 적용 가이드

**하이브리드 검색 사용 (권장 - 90% 케이스):**
- ✅ 일반적인 사용자 질문 (예: "에이전트 보안 강화 방법")
- ✅ 복합 쿼리 (전문 용어 + 자연어)
- ✅ 프로덕션 RAG 시스템
- ✅ 정확도가 중요한 경우

**벡터 전용 검색 사용:**
- ✅ 의미적 유사성이 중요한 경우 (예: FAQ, 유사 문서 찾기)
- ✅ 다국어 검색 (한국어 쿼리로 영어 문서 검색)
- ✅ 개념적 질문 (예: "비용 절감 방법" ≈ "예산 최적화")

**키워드 전용 검색 사용:**
- ✅ 코드 스니펫 검색 (예: "pip install azure-search")
- ✅ 정확한 명령어/API 이름 검색
- ✅ 약어 검색 (예: "MCP", "RAG", "LLM")
- ✅ 속도가 매우 중요한 경우

---

#### 🔬 추가 실험 아이디어

다양한 쿼리로 테스트해보세요:

**1. 전문 용어 쿼리:**
```python
test_query = "MCP 서버 개발"  # 키워드 검색 유리
```

**2. 자연어 질문:**
```python
test_query = "에이전트를 안전하게 만드는 방법은?"  # 벡터 검색 유리
```

**3. 다국어 쿼리:**
```python
test_query = "Azure AI agent deployment"  # 하이브리드 최적
```

**4. 약어 쿼리:**
```python
test_query = "RAG 성능 최적화"  # 하이브리드 최적
```

각 쿼리별로 3가지 방식의 결과를 비교하면 **검색 엔진 선택 기준**을 명확히 이해할 수 있습니다! 🚀

## 10. 설정 파일 업데이트 (Update Configuration)

인덱스 이름을 `config.json`에 저장하여 Notebook 3 (에이전트 배포)에서 사용할 수 있도록 합니다.

In [13]:
# config.json 다시 로드
with open("./config.json", "r", encoding="utf-8") as f:
    config = json.load(f)

# 인덱스 이름 추가
config["search_index"] = index_name

# 업데이트된 설정 저장
with open("./config.json", "w", encoding="utf-8") as f:
    json.dump(config, f, indent=2, ensure_ascii=False)

print("✅ 설정 파일 업데이트 완료!")
print(f"   - 인덱스 이름: {index_name}")
print(f"   - 저장 위치: config.json")
print("\n🔗 다음 단계:")
print("   - Notebook 3에서 이 설정을 사용하여 Research Agent 배포")
print("   - Research Agent가 이 인덱스를 검색하여 질문에 답변합니다")

✅ 설정 파일 업데이트 완료!
   - 인덱스 이름: ai-agent-knowledge-base
   - 저장 위치: config.json

🔗 다음 단계:
   - Notebook 3에서 이 설정을 사용하여 Research Agent 배포
   - Research Agent가 이 인덱스를 검색하여 질문에 답변합니다


## 11. RAG 구성 완료 및 요약 (RAG Setup Summary)

### 🎉 Lab 2 완료!

이번 실습에서 구성한 내용:

| 항목 | 세부 내용 |
|------|-----------|
| **인덱스 이름** | `ai-agent-knowledge-base` |
| **문서 수** | 54개 (agent development, RAG patterns, MCP, deployment, multi-agent) |
| **벡터 차원** | 3072 (text-embedding-3-large, 기본값) |
| **검색 알고리즘** | HNSW (Hierarchical Navigable Small World) |
| **언어 분석기** | ko.microsoft (한국어 지원) |
| **검색 방식** | 하이브리드 (벡터 + 키워드) |

### 📊 RAG 파이프라인 구성 완료

```
📄 Knowledge Base (54 docs)
    ↓
🔢 Embeddings (3072-dim vectors)
    ↓
💾 Azure AI Search Index
    ↓
🔍 Hybrid Search (Vector + Keyword)
    ↓
🤖 Research Agent (Notebook 3에서 배포 예정)
```

---

### ⚡ 성능 벤치마크 및 최적화 가이드

#### 📈 검색 성능 지표 (문서 수별)

**테스트 환경:**
- Azure AI Search: Standard S1 tier
- 임베딩: text-embedding-3-large (3072 dim)
- 쿼리: 일반적인 자연어 질문

| 문서 수 | 벡터 검색 | 키워드 검색 | 하이브리드 검색 | 인덱싱 시간 |
|---------|----------|------------|---------------|------------|
| **100개** | 50-80ms | 10-20ms | 60-100ms | ~30초 |
| **1,000개** | 80-120ms | 15-30ms | 100-150ms | ~5분 |
| **10,000개** | 150-250ms | 30-50ms | 180-300ms | ~30분 |
| **100,000개** | 300-500ms | 50-100ms | 350-600ms | ~5시간 |

**이 Lab (54개 문서):**
- 예상 검색 시간: **50-100ms** (하이브리드)
- 인덱싱 시간: **~20초**

---

#### 🆚 검색 방식별 성능 비교

**1. 정확도 (Precision@5)**

| 쿼리 유형 | 벡터 전용 | 하이브리드 | 키워드 전용 |
|----------|----------|-----------|------------|
| 자연어 질문 | 75-85% | **90-95%** ⭐ | 60-70% |
| 전문 용어 | 65-75% | **85-95%** ⭐ | 80-90% |
| 코드/명령어 | 50-60% | **75-85%** ⭐ | 85-95% |
| 다국어 쿼리 | 80-90% | **90-95%** ⭐ | 40-50% |

**결론:** 하이브리드 검색이 모든 쿼리 유형에서 **가장 높은 정확도**

---

**2. 속도 (ms per query)**

| 문서 수 | 벡터 전용 | 하이브리드 | 키워드 전용 |
|---------|----------|-----------|------------|
| 100 | 50ms | 60ms | **10ms** ⭐ |
| 1,000 | 80ms | 100ms | **15ms** ⭐ |
| 10,000 | 150ms | 180ms | **30ms** ⭐ |
| 100,000 | 300ms | 350ms | **50ms** ⭐ |

**결론:** 키워드 검색이 **가장 빠름**, 하이브리드는 20-30% 느림 (여전히 실시간)

---

**3. 비용 (검색 1,000회 기준)**

| 항목 | 벡터 전용 | 하이브리드 | 키워드 전용 |
|------|----------|-----------|------------|
| **쿼리 임베딩** | $0.13/1M tokens ≈ $0.01 | $0.13/1M tokens ≈ $0.01 | $0 (불필요) |
| **검색 비용** | 포함 (Azure AI Search 요금) | 포함 | 포함 |
| **총 비용 차이** | +$0.01 | +$0.01 | **$0** ⭐ |

**결론:** 하이브리드/벡터 검색의 추가 비용은 **미미함** (1,000회당 $0.01)

---

#### 💡 프로덕션 환경 최적화 가이드

**1️⃣ 문서 수에 따른 전략**

**소규모 (< 1,000개 문서)**
- ✅ 하이브리드 검색 사용
- ✅ HNSW 알고리즘 기본 설정
- ✅ Standard S1 tier 충분
- ⏱️ 검색 시간: 50-100ms

**중규모 (1,000 ~ 10,000개 문서)**
- ✅ 하이브리드 검색 사용
- ✅ HNSW efSearch 파라미터 조정 고려
- ✅ Standard S2 tier 권장
- ⏱️ 검색 시간: 100-200ms

**대규모 (10,000+ 문서)**
- ✅ 하이브리드 검색 사용
- ⚙️ IVF 알고리즘 고려 (속도 우선 시)
- ✅ Standard S3+ tier 필요
- ✅ 복제본(Replica) 추가로 부하 분산
- ⏱️ 검색 시간: 200-400ms

---

**2️⃣ 티어별 성능 비교**

| 티어 | 최대 문서 수 | 검색 단위(SU) | 복제본 | 파티션 | 비용/월 |
|------|-------------|--------------|--------|--------|---------|
| **Free** | 10,000 | 1 | 1 | 1 | $0 |
| **Basic** | 1,000,000 | 3 | 3 | 1 | ~$75 |
| **Standard S1** | 15,000,000 | 36 | 12 | 12 | ~$250 |
| **Standard S2** | 60,000,000 | 36 | 12 | 12 | ~$1,000 |

**이 Lab 권장:** Basic 또는 Standard S1

---

**3️⃣ 검색 품질 최적화**

**임베딩 모델 선택:**
```python
# 정확도 우선 (프로덕션 권장)
model = "text-embedding-3-large"  # 3072 dim
dimensions = 3072
# → Precision@5: 90-95%

# 비용/속도 우선 (프로토타입)
model = "text-embedding-3-small"  # 1536 dim
dimensions = 1536
# → Precision@5: 85-90% (정확도 약간 감소)
```

**문서 청킹 전략:**
```python
# ✅ 좋은 예: 섹션별 분할 (이 Lab에서 사용)
# 각 섹션이 독립적 의미 단위
# → 검색 품질 향상

# ⚠️ 대안: 고정 길이 청킹
# 512 토큰씩 분할, 100 토큰 오버랩
# → 구현 간단, 품질 약간 감소
```

---

**4️⃣ 비용 최적화**

**인덱싱 비용:**
```
문서 1개당 비용 = 임베딩 비용 + 저장 비용

임베딩 비용 (text-embedding-3-large):
- 평균 500 토큰/문서 기준
- 54 문서: 27,000 토큰 ≈ $0.0035
- 10,000 문서: 5M 토큰 ≈ $0.65

저장 비용 (Standard S1):
- 기본 요금: $250/월 (15M 문서까지)
- 추가 저장 공간: 불필요
```

**검색 비용 절감 팁:**
1. **캐싱**: 동일 쿼리 결과를 메모리에 캐싱 (Redis 등)
2. **쿼리 임베딩 재사용**: 유사 쿼리는 임베딩 공유
3. **배치 임베딩**: Azure OpenAI Batch API 사용 (50% 할인)

---

#### 🎯 프로덕션 체크리스트

**✅ 필수 설정**
- [ ] 하이브리드 검색 활성화
- [ ] 한국어 분석기 설정 (`ko.microsoft`)
- [ ] HNSW 알고리즘 사용
- [ ] 3072 차원 벡터 (정확도 우선)
- [ ] Managed Identity 인증 (보안)

**✅ 성능 최적화**
- [ ] 복제본(Replica) 추가 (고가용성)
- [ ] 파티션(Partition) 추가 (대량 데이터)
- [ ] 모니터링 설정 (Azure Monitor)
- [ ] 알림 규칙 설정 (검색 지연 시)

**✅ 비용 최적화**
- [ ] 불필요한 필드 제거 (저장 공간 절약)
- [ ] 쿼리 캐싱 구현
- [ ] 배치 임베딩 사용 (초기 인덱싱)
- [ ] 예약 용량 구매 고려 (20% 할인)

**✅ 보안**
- [ ] Private Endpoint 설정 (VNet)
- [ ] Firewall 규칙 설정
- [ ] RBAC 권한 최소화
- [ ] 감사 로깅 활성화

---

### 🔗 다음 단계: Notebook 3

**Lab 3: Multi-Agent 시스템 배포하기**에서는:

1. **MCP Server 배포** - 날씨 정보 도구 서버 (Container Apps)
2. **Agent Container 빌드** - Main/Tool/Research Agent 컨테이너화
3. **Agent Service 배포** - Multi-Agent 시스템 배포 (Container Apps)
4. **Managed Identity 권한 설정** - Azure AI User 역할 자동 할당
5. **HTTP API 테스트** - 배포된 Agent 엔드포인트 테스트
6. **Application Analytics 확인** - 메트릭 및 성능 모니터링
7. **Tracing UI 확인** - 상세 실행 흐름 및 Input/Output 확인

**Research Agent가 이 인덱스(`ai-agent-knowledge-base`)를 검색하여 사용자 질문에 답변합니다!**

이제 Notebook 3을 실행할 준비가 완료되었습니다! 🚀

## 12. 문제 해결 (Troubleshooting)

### ❌ 일반적인 오류와 해결 방법

#### 1️⃣ **인덱스 생성 실패 - Forbidden 오류**
```
HttpResponseError: Operation returned an invalid status 'Forbidden'
```
**해결 방법:**
- 섹션 4에서 역할 할당이 완료되었는지 확인
- 역할 전파를 위해 1-2분 대기 후 재시도
- Azure Portal에서 수동으로 역할 확인: AI Search 서비스 → Access Control (IAM)

#### 2️⃣ **인덱스 생성 실패 - 일반 오류**
```
Error: Index creation failed
```
**해결 방법:**
- Azure AI Search 서비스가 정상 실행 중인지 확인
- 서비스 티어가 벡터 검색을 지원하는지 확인 (Basic 이상)
- `index_name`이 Azure 명명 규칙을 따르는지 확인 (소문자, 숫자, 하이픈만 사용)

#### 3️⃣ **임베딩 생성 실패**
```
Error: Rate limit exceeded / API quota exceeded
```
**해결 방법:**
- `time.sleep(0.5)` 대신 더 긴 대기 시간 설정 (예: 1초)
- Azure OpenAI 서비스의 TPM (Tokens Per Minute) 할당량 확인
- 배포 모델의 용량 확대 필요 시 Azure Portal에서 조정

#### 4️⃣ **문서 업로드 실패**
```
Error: Document upload failed
```
**해결 방법:**
- `contentVector` 차원이 정확히 3072인지 확인
- 모든 필수 필드(`id`, `title`, `content`)가 있는지 확인
- 문서 ID가 고유한지 확인 (중복 시 오류 발생)

#### 5️⃣ **검색 결과가 없음**
```
Search returned 0 results
```
**해결 방법:**
- 문서가 실제로 업로드되었는지 확인: `search_client.get_document_count()`
- 인덱싱이 완료될 때까지 1-2분 대기
- 쿼리 텍스트가 너무 구체적이지 않은지 확인

#### 6️⃣ **한국어 검색 품질 낮음**
```
Korean text search returns poor results
```
**해결 방법:**
- 인덱스의 `content` 필드가 `ko.microsoft` 분석기를 사용하는지 확인
- 하이브리드 검색을 사용하여 벡터 + 키워드 검색 결합
- 쿼리 텍스트를 더 자연스러운 문장으로 작성

### 🔍 디버깅 팁

```python
# 인덱스 문서 수 확인
doc_count = search_client.get_document_count()
print(f"인덱스의 총 문서 수: {doc_count}")

# 특정 문서 조회
doc = search_client.get_document(key="doc_001")
print(f"문서 제목: {doc['title']}")
print(f"벡터 차원: {len(doc['contentVector'])}")

# 인덱스 통계 확인 (Azure Portal)
# https://portal.azure.com → AI Search 서비스 → 인덱스 → 통계 탭
```